In [1]:
import os
import urllib.request

# Create directories to store data
os.makedirs('data/images', exist_ok=True)

# Download dataset (example: CelebA dataset)
url = 'https://s3.amazonaws.com/fast-ai-imageclas/imagenette2-160.tgz'
urllib.request.urlretrieve(url, 'data/images/imagenette2-160.tgz')

# Extract dataset
import tarfile
with tarfile.open('data/images/imagenette2-160.tgz', 'r:gz') as tar: 
    tar.extractall('data/images/')


IMAGE PREPROCESSING

In [2]:
from PIL import Image
import numpy as np
import cv2

def preprocess_image(image_path, target_size=(224, 224)):
    image = Image.open(image_path)
    image = image.resize(target_size)
    image = np.array(image) / 255.0  # Normalize pixel values
    return image

image_path = 'data/images/imagenette2-160/val/n01440764/ILSVRC2012_val_00009111.JPEG'
preprocessed_image = preprocess_image(image_path)


In [7]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

base_model = MobileNetV2(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(2, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze pre-trained layers
for layer in base_model.layers:
    layer.trainable = False





9406464/9406464 [==============================] - 2s 0us/step


In [3]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Assuming there are 10 classes in your dataset
num_classes = 10

base_model = MobileNetV2(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(num_classes, activation='softmax')(x)  # Adjust number of units here
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze pre-trained layers
for layer in base_model.layers:
    layer.trainable = False





9406464/9406464 [==============================] - 3s 0us/step


In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    'data/images/imagenette2-160/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_generator, epochs=10, steps_per_epoch=len(train_generator))

Found 9469 images belonging to 10 classes.

Epoch 1/10


296/296 [==============================] - 200s 666ms/step - loss: 0.4180 - accuracy: 0.8808
Epoch 2/10
296/296 [==============================] - 145s 490ms/step - loss: 0.1811 - accuracy: 0.9445
Epoch 3/10
296/296 [==============================] - 144s 486ms/step - loss: 0.1603 - accuracy: 0.9464
Epoch 4/10
296/296 [==============================] - 143s 482ms/step - loss: 0.1314 - accuracy: 0.9580
Epoch 5/10
296/296 [==============================] - 141s 477ms/step - loss: 0.1261 - accuracy: 0.9590
Epoch 6/10
296/296 [==============================] - 145s 488ms/step - loss: 0.1161 - accuracy: 0.9608
Epoch 7/10
296/296 [==============================] - 140s 474ms/step - loss: 0.1194 - accuracy: 0.9610
Epoch 8/10
296/296 [==============================] - 141s 476ms/step - loss: 0.1098 - accuracy: 0.9636
Epoch 9/10
296/296 [==============================] - 138s 465ms/step - loss: 0.1073 - accuracy: 0.9632
Epoch 10/10
296/29

In [5]:
validation_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow_from_directory(
    'data/images/imagenette2-160/val',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

loss, accuracy = model.evaluate(validation_generator)
print(f'Validation Accuracy: {accuracy}')

Found 3925 images belonging to 10 classes.
123/123 [==============================] - 56s 449ms/step - loss: 0.0970 - accuracy: 0.9687
Validation Accuracy: 0.968662440776825


MODEL COMPILATION


In [6]:
from tensorflow.keras.layers import Dense

# Define model architecture with separate output layers for gender and age
base_model = MobileNetV2(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
gender_predictions = Dense(2, activation='softmax', name='gender_predictions')(x)  # Output layer for gender
age_predictions = Dense(5, activation='softmax', name='age_predictions')(x)  # Output layer for age
model = Model(inputs=base_model.input, outputs=[gender_predictions, age_predictions])

# Freeze pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model with appropriate loss functions for gender and age
model.compile(optimizer='adam',
              loss={'gender_predictions': 'categorical_crossentropy', 'age_predictions': 'categorical_crossentropy'},
              metrics=['accuracy'])


GENDER PREDICTION

In [16]:
from tensorflow.keras.preprocessing import image
import numpy as np


# Load and preprocess the image
img_path = r'D:\Int Project\BASIC\Gender-and-Age-Detection-master\Gender-and-Age-Detection-master\man2.jpg'  # Update with the path to your image
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.  # Normalization


# Make predictions
gender_predictions, age_predictions = model.predict(img_array)


# Get predicted gender and age class indices
predicted_gender_index = np.argmax(gender_predictions)
predicted_age_index = np.argmax(age_predictions)


# Assuming you have dictionaries mapping class indices to class labels for gender and age
gender_labels = {0: 'Female', 1: 'Male'}
age_labels = {0: 'Senior', 1: 'Adult', 2: 'Young Adult', 3: 'Teenager', 4: 'Child'}


# Get predicted gender and age labels
predicted_gender_label = gender_labels.get(predicted_gender_index, "Unknown Gender")
predicted_age_label = age_labels.get(predicted_age_index, "Unknown Age")


# Print predicted gender and age
print("Predicted Gender:", predicted_gender_label)
print("Predicted Age:", predicted_age_label)

1/1 [==============================] - 0s 40ms/step
Predicted Gender: Male
Predicted Age: Adult
